<a href="https://colab.research.google.com/github/katusabevictoria/katusabevictoria/blob/main/Ensemble_Trees_Exercise__Victoria_Katusabe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor # NEW
from sklearn import set_config
set_config(transform_output='pandas')



1. Train and evaluate a default Bagged Trees

In [7]:
#Add custom functions (from snippets if you have saved it)

def regression_metrics(y_true, y_pred, label='', verbose = True, output_dict=False):
  # Get metrics
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = mean_squared_error(y_true, y_pred, squared=False)
  r_squared = r2_score(y_true, y_pred)
  if verbose == True:
    # Print Result with Label and Header
    header = "-"*60
    print(header, f"Regression Metrics: {label}", header, sep='\n')
    print(f"- MAE = {mae:,.3f}")
    print(f"- MSE = {mse:,.3f}")
    print(f"- RMSE = {rmse:,.3f}")
    print(f"- R^2 = {r_squared:,.3f}")
  if output_dict == True:
      metrics = {'Label':label, 'MAE':mae,
                 'MSE':mse, 'RMSE':rmse, 'R^2':r_squared}
      return metrics

def evaluate_regression(reg, X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False):
  # Get predictions for training data
  y_train_pred = reg.predict(X_train)

  # Call the helper function to obtain regression metrics for training data
  results_train = regression_metrics(y_train, y_train_pred, verbose = verbose,
                                     output_dict=output_frame,
                                     label='Training Data')
  print()
  # Get predictions for test data
  y_test_pred = reg.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = regression_metrics(y_test, y_test_pred, verbose = verbose,
                                  output_dict=output_frame,
                                    label='Test Data' )

  # Store results in a dataframe if ouput_frame is True
  if output_frame:
    results_df = pd.DataFrame([results_train,results_test])
    # Set the label as the index
    results_df = results_df.set_index('Label')
    # Set index.name to none to get a cleaner looking result
    results_df.index.name=None
    # Return the dataframe
    return results_df.round(3)

In [8]:
## Loading data from published google sheet
fpath ="/content/drive/MyDrive/Colab data uploads/Boston_Housing_from_Sklearn - Boston_Housing_from_Sklearn.csv"
df = pd.read_csv(fpath)
df

,CRIM,NOX,RM,AGE,PTRATIO,LSTAT,PRICE
0,0.00632,0.538,6.575,65.2,15.3,4.98,24.0
1,0.02731,0.469,6.421,78.9,17.8,9.14,21.6
2,0.02729,0.469,7.185,61.1,17.8,4.03,34.7
3,0.03237,0.458,6.998,45.8,18.7,2.94,33.4
4,0.06905,0.458,7.147,54.2,18.7,5.33,36.2
...,...,...,...,...,...,...,...
501,0.06263,0.573,6.593,69.1,21.0,9.67,22.4
502,0.04527,0.573,6.120,76.7,21.0,9.08,20.6
503,0.06076,0.573,6.976,91.0,21.0,5.64,23.9
504,0.10959,0.573,6.794,89.3,21.0,6.48,22.0


In [9]:
# Make X and y
target = 'PRICE'
X = df.drop(columns=target).copy()
y = df[target].copy()

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
# Instatiate Default model
default = DecisionTreeRegressor(random_state=42)

# Fit default model on training data only
default.fit(X_train, y_train)

# Evaluate with custom function
evaluate_regression(default, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 0.000
- MSE = 0.000
- RMSE = 0.000
- R^2 = 1.000

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 3.141
- MSE = 26.658
- RMSE = 5.163
- R^2 = 0.619


In [11]:
# Get depth of tree with default model
depth_used = default.get_depth()
depth_used

20

In [12]:
# Always helpful to see available paramters
default.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

In [13]:
# Define Param Grid
param_grid = {'max_depth' : (np.arange(5,depth_used,1)),
              'min_samples_leaf' : np.arange(1,11)}

2. Use GridSearchCV to tune the Bagged Tree model to optimize performance on the test set.

Evaluate the best Bagged Tree model's performance.


In [14]:
# Instantiate GridSearchCV
grid_search = GridSearchCV(default, param_grid, n_jobs = -1, verbose = 1)

In [15]:
# Fit the Gridsearch on the training data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


GridSearchCV(estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
                         'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
             verbose=1)

In [16]:
# Obtain best values
grid_search.best_params_

{'max_depth': 8, 'min_samples_leaf': 7}

In [17]:
# Define the best version of the model already refitted on entire training set
best_model = grid_search.best_estimator_

In [18]:
# Use custom function to evalute the model
evaluate_regression( best_model, X_train, y_train, X_test, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 2.221
- MSE = 11.803
- RMSE = 3.436
- R^2 = 0.867

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 2.613
- MSE = 16.046
- RMSE = 4.006
- R^2 = 0.771


3. Train and evaluate a default random forest

In [23]:
# Saving list of categorical columns
cat_cols = X_train.select_dtypes('object').columns
# Constructing categorical preprocessing objects
cat_imputer = SimpleImputer(strategy='constant', fill_value='MISSING')
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_pipe = make_pipeline(cat_imputer,ohe_encoder)
cat_tuple = ('cat',cat_pipe, cat_cols)
cat_tuple



('cat',
 Pipeline(steps=[('simpleimputer',
                  SimpleImputer(fill_value='MISSING', strategy='constant')),
                 ('onehotencoder',
                  OneHotEncoder(handle_unknown='ignore', sparse_output=False))]),
 Index([], dtype='object'))

In [25]:
# Save list of numeric columns
num_cols = X_train.select_dtypes('number').columns
# Constructing numeric preprocesssing objects
num_imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()
num_pipe = make_pipeline(num_imputer, scaler)
num_tuple = ('num',num_pipe, num_cols)
num_tuple



('num',
 Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                 ('standardscaler', StandardScaler())]),
 Index(['CRIM', 'NOX', 'RM', 'AGE', 'PTRATIO', 'LSTAT'], dtype='object'))

In [27]:
# Define a column transformer
preprocessor  = ColumnTransformer([num_tuple, cat_tuple],
                                  verbose_feature_names_out=False)



4. Use GridSearchCV to tune the Random Forest model to optimize performance on the test set.



In [28]:
# Instantiate default random forest model
rf = RandomForestRegressor(random_state = 42)
# Model Pipeline
rf_pipe = make_pipeline(preprocessor, rf)


In [29]:
# Fit the model pipeline on the training data only
rf_pipe.fit(X_train, y_train)



Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['CRIM', 'NOX', 'RM', 'AGE', 'PTRATIO', 'LSTAT'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index([], dtype='object'))],
                                   verbose_feature_names_out=False)),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [30]:
# Use custom function to evaluate default model
evaluate_regression(rf_pipe, X_train, y_train, X_test, y_test)



------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 0.953
- MSE = 2.026
- RMSE = 1.423
- R^2 = 0.977

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 2.207
- MSE = 11.643
- RMSE = 3.412
- R^2 = 0.834


6. Explain in a text cell how your model will perform if deployed by referring to the metrics. Ex. How close can your stakeholders expect its predictions to be to the true value?


When deployed, the performance of my model can be evaluated using various metrics depending on the nature of the problem. For regression tasks, stakeholders can expect predictions to be evaluated based on metrics such as Mean Absolute Error (MAE), Mean Squared Error (MSE), or Root Mean Squared Error (RMSE). These metrics quantify how close the predictions are to the true values.

For classification tasks, metrics like accuracy, precision, recall, and F1-score are commonly used to evaluate model performance. Accuracy measures the proportion of correct predictions, while precision measures the proportion of true positive predictions among all positive predictions. Recall measures the proportion of true positive predictions among all actual positives. The F1-score is the harmonic mean of precision and recall, providing a balanced measure of a model's performance.

Stakeholders can expect the model's performance to be assessed using these metrics, providing them with insights into the model's accuracy, reliability, and effectiveness in making predictions or classifications. Additionally, stakeholders should consider the specific context of their problem and choose evaluation metrics that align with their goals and priorities.